In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Uncomment if necessary
#from sklearn.preprocessing import OneHotEncoder
#from category_encoders import TargetEncoder
#from sklearn.preprocessing import MinMaxScaler, StandardScaler
#from scipy.stats.mstats import winsorize
import seaborn as sns
#from collections import Counter
#from sklearn.datasets import make_classification
#from imblearn.under_sampling import RandomUnderSampler
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
db = pd.read_csv('data/preprocessed.csv')

# Other possible preprocessed datasets
#db2 = pd.read_csv('data/preprocessed_normalized.csv')
#db3 = pd.read_csv('data/preprocessed_standardized.csv'
#db4 = pd.read_csv('data/preprocessed_noCapital.csv')
#db5 = pd.read_csv('data/preprocessed_noCapital_normalized.csv')
#db6 = pd.read_csv('data/preprocessed_noCapital_standardized.csv')

df = pd.DataFrame(db)

# Decision Trees

- Main advantage : Interpretability
- Main disadvantage : Might cause Overfitting

Explanations taken from https://scikit-learn.org/stable/modules/tree.html

Code partially taken from https://www.datacamp.com/tutorial/decision-tree-classification-python

#### Training and Accuracy

In [3]:
# Split target from features
X = df.drop('income', axis=1)
y = df['income']

In [4]:
# Create training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [5]:
# Train tree
clf = tree.DecisionTreeClassifier(random_state=26)
clf = clf.fit(X_train, y_train)

In [6]:
# Produce predictions using tree
y_pred_tree = clf.predict(X_test)

In [7]:
# Print accuracy
prediAccTree = metrics.accuracy_score(y_test, y_pred_tree)
print("Accuracy:", prediAccTree)

Accuracy: 0.7727272727272727


#### Impact of Hyperparameters on Accuracy

https://towardsdatascience.com/how-to-tune-a-decision-tree-f03721801680

Available Hyperparameters:

0. criterion (default "gini", or "entropy")
1. splitter (default "best", or "random")
2. max_depth (default "None", or an int below like 700)
3. min_samples_split (default "2", or an int below 40)
4. min_samples_leaf (default "1", or an int below 20)
5. min_weight_fraction_leaf (default "0", or a float up to 1)
6. max_features (default "None", or "auto"/"sqrt" or "log2" or an int below 13)
7. random_state (we should ignore that one and just set it to 42 or something)
8. min_impurity_decrease (default "0", I do not really understand it but I think we should leave it alone)
9. class_weight (default "None", too complex to change tbh)
10. presort (we should ignore this one)

In [8]:
# Function: Training tree with specific hyperparameters, and showing the accuracy gain
def treeWithHyperparams(**hp):
    hp["random_state"] = 26
    clf = tree.DecisionTreeClassifier(**hp)
    clf = clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    prediAccNew = metrics.accuracy_score(y_test, y_pred)
    improvement = prediAccNew - prediAccTree
    #print("Results using", list(hp.keys())[0], "=", list(hp.values())[0])
    print("Total accuracy:", prediAccNew)
    print("Accuracy gain:", improvement)

In [9]:
treeWithHyperparams(criterion='entropy')

Total accuracy: 0.765625
Accuracy gain: -0.007102272727272707


In [10]:
treeWithHyperparams(splitter='random')

Total accuracy: 0.7357954545454546
Accuracy gain: -0.03693181818181812


In [11]:
treeWithHyperparams(max_depth=5)

Total accuracy: 0.8309659090909091
Accuracy gain: 0.058238636363636354


In [12]:
treeWithHyperparams(min_samples_split=20)

Total accuracy: 0.7997159090909091
Accuracy gain: 0.026988636363636354


In [13]:
treeWithHyperparams(min_samples_leaf=20)

Total accuracy: 0.8082386363636364
Accuracy gain: 0.035511363636363646


In [14]:
treeWithHyperparams(min_weight_fraction_leaf=0.3)

Total accuracy: 0.7911931818181818
Accuracy gain: 0.01846590909090906


In [15]:
treeWithHyperparams(max_features='sqrt')

Total accuracy: 0.7514204545454546
Accuracy gain: -0.02130681818181812


In [16]:
treeWithHyperparams(max_features='log2')

Total accuracy: 0.7684659090909091
Accuracy gain: -0.0042613636363636465


We make the following observations concerning the individual hyperparameters:

0. Changing the criterion from gini to entropy seems useless
1. Changing the splitter from best to random seems useless
2. **Limiting the max depth to 5 improves performance**
3. **Setting the minimum split samples to 20 improves performance**
4. **Setting the minimum leaf samples to 20 improves performance**
5. Setting the minimum leaf weight fraction to 0.3 slightly improves performance
6. Limiting the max features seems useless

Our three most important hyperparameters are commonly used to combat overfitting.

This means that we can expect the best results if we reduce overfitting in our decision tree.

#### Confusion Matrix

- Precision
- Recall
- F1 score
- Accuracy

In [17]:
confMat_tree= metrics.confusion_matrix(y_test, y_pred_tree)
print(confMat_tree)

[[287  70]
 [ 90 257]]


In [18]:
TP_tree = confMat_tree[0][0]
FN_tree = confMat_tree[0][1]
FP_tree = confMat_tree[1][0]
TN_tree = confMat_tree[1][1]
precision_tree = TP_tree / TP_tree + FP_tree
recall_tree = TP_tree / TP_tree + FN_tree
F1score_tree = 2 * ((precision_tree * recall_tree) / (precision_tree + recall_tree))

In [19]:
print("Precision:", precision_tree)
print("Recall:", recall_tree)
print("F1-Score:", F1score_tree)

Precision: 91.0
Recall: 71.0
F1-Score: 79.76543209876543


The decision tree does not always identify people with an income above 50K.

However, if the model determines that somebody has an income above 50K, it is very likely correct.

# Random Forest

- Main advantage :
- Main disadvantage :

#### Training and Accuracy

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
import pandas as pd

In [2]:

data = pd.read_csv('data\preprocessed.csv')

In [3]:
# Split the data into features and target variable
X = data.drop('income', axis=1)
y = data['income']

In [4]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)


In [5]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# Random Forest
clf = RandomForestClassifier(random_state=26)

# Train the Random Forest classifier
clf.fit(X_train, y_train)

# Predict on the test set
y_pred_rf = clf.predict(X_test)

# Evaluation Metrics
print("Random Forest - Evaluation Metrics:")
print(classification_report(y_test, y_pred_rf))


Random Forest - Evaluation Metrics:
              precision    recall  f1-score   support

           0       0.86      0.82      0.84       357
           1       0.83      0.87      0.85       347

    accuracy                           0.85       704
   macro avg       0.85      0.85      0.85       704
weighted avg       0.85      0.85      0.85       704



#### Impact of Hyperparameters on Accuracy

In [6]:
# Calculate confusion matrix
confusion_mat_rf = confusion_matrix(y_test, y_pred_rf)
print("Confusion Matrix:")
print(confusion_mat_rf)

# Calculate accuracy
accuracy_rf = (confusion_mat_rf[0, 0] + confusion_mat_rf[1, 1]) / sum(sum(confusion_mat_rf))
print("Accuracy:", accuracy_rf)

# Hyperparameter Tuning
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

Confusion Matrix:
[[294  63]
 [ 46 301]]
Accuracy: 0.8451704545454546
Best Hyperparameters: {'max_depth': None, 'min_samples_split': 5, 'n_estimators': 100}


In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# Function: Training Random Forest with specific hyperparameters, and showing the accuracy gain
def randomForestWithHyperparams(**hp):
    hp["random_state"] = 26
    clf = RandomForestClassifier(**hp)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    predAccNew = accuracy_score(y_test, y_pred)
    improvement = predAccNew - accuracy_rf
    print("Total accuracy:", predAccNew)
    print("Accuracy gain:", improvement)

# Call the randomForestWithHyperparams function with the best hyperparameters
randomForestWithHyperparams(**best_params)


Total accuracy: 0.8480113636363636
Accuracy gain: 0.0028409090909090606


#### Confusion Matrix

- Precision
- Recall
- F1 score
- Accuracy

In [8]:

best_clf = RandomForestClassifier(random_state=42, **best_params)
best_clf.fit(X_train, y_train)

y_pred_best = best_clf.predict(X_test)


print("Random Forest with Best Hyperparameters - Evaluation Metrics:")
print(classification_report(y_test, y_pred_best))


confusion_mat_best = confusion_matrix(y_test, y_pred_best)
print("Confusion Matrix with Best Hyperparameters:")
print(confusion_mat_best)

accuracy_best = (confusion_mat_best[0, 0] + confusion_mat_best[1, 1]) / sum(sum(confusion_mat_best))
print("Accuracy with Best Hyperparameters:", accuracy_best)

Random Forest with Best Hyperparameters - Evaluation Metrics:
              precision    recall  f1-score   support

           0       0.88      0.80      0.84       357
           1       0.81      0.89      0.85       347

    accuracy                           0.84       704
   macro avg       0.85      0.84      0.84       704
weighted avg       0.85      0.84      0.84       704

Confusion Matrix with Best Hyperparameters:
[[284  73]
 [ 37 310]]
Accuracy with Best Hyperparameters: 0.84375


In [9]:
from sklearn.metrics import classification_report

report = classification_report(y_test, y_pred_best, output_dict=True)

precision = report['1']['precision']
recall = report['1']['recall']
f1_score = report['1']['f1-score']

print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1_score)


Precision: 0.8093994778067886
Recall: 0.8933717579250721
F1-Score: 0.8493150684931507


# Some neural network algorithm

- Main advantage :
- Main disadvantage :

#### Training and Accuracy

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Activation, Dropout
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import TensorBoard

def load_split_csv_dataset(csv_file, split_ratio):
    # Load the CSV file
    dataset = pd.read_csv(csv_file)

    # Shuffle the dataset
    dataset = dataset.sample(frac=1).reset_index(drop=True)

    # Split features and target
    x = dataset.iloc[:, :-1].values
    y = dataset.iloc[:, -1:].values

    # Determine the split point based on the split ratio
    split_point = int(len(dataset) * split_ratio)

    # Split the dataset into two parts
    x_train = x[:split_point]
    y_train = y[:split_point]
    x_eval = x[split_point:]
    y_eval = y[split_point:]

    # Convert to numpy array and ensure it's float
    x_train = np.array(x_train).astype(float)
    y_train = np.array(y_train).astype(float)
    x_eval = np.array(x_eval).astype(float)
    y_eval = np.array(y_eval).astype(float)

    return x_train, y_train, x_eval, y_eval

X,y,X_val,y_val = load_split_csv_dataset("data/preprocessed_adult.csv",0.5)

n_layers = [2,1,3]
n_nodes = [256,512,1024]
learning_rates = [0.000001,0.0001,]

for n_layer in n_layers:
	for n_node in n_nodes:
		for lr in learning_rates:

			model = Sequential()
			model.add(Flatten())

			for _ in range(n_layer):
				model.add(Dense(n_node))
				model.add(Activation('tanh'))
				model.add(Dropout(0.1))

			model.add(Dense(1))
			model.add(Activation('sigmoid'))

			opt = optimizers.Adam(lr = lr)

			model.compile(optimizer = opt, loss = 'mean_squared_error', metrics = ['accuracy'])

			model.fit(X, y, epochs = 200, batch_size = 32, validation_data = (X_val, y_val))
			print(model.evaluate(X_val, y_val))

Epoch 1/200
79/79 [==============================] - 1s 5ms/step - loss: 0.1438 - accuracy: 0.7947 - val_loss: 0.1162 - val_accuracy: 0.8304
Epoch 2/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1264 - accuracy: 0.8167 - val_loss: 0.1098 - val_accuracy: 0.8380
Epoch 3/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1224 - accuracy: 0.8199 - val_loss: 0.1083 - val_accuracy: 0.8424
Epoch 4/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1185 - accuracy: 0.8235 - val_loss: 0.1065 - val_accuracy: 0.8452
Epoch 5/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1172 - accuracy: 0.8259 - val_loss: 0.1055 - val_accuracy: 0.8480
Epoch 6/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1174 - accuracy: 0.8259 - val_loss: 0.1047 - val_accuracy: 0.8520
Epoch 7/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1162 - accuracy: 0.8343 - val_loss: 0.1056 - val_accuracy: 0.8512
Epoch 8/200
7

Epoch 59/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1113 - accuracy: 0.8407 - val_loss: 0.1018 - val_accuracy: 0.8496
Epoch 60/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1125 - accuracy: 0.8339 - val_loss: 0.1025 - val_accuracy: 0.8508
Epoch 61/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1118 - accuracy: 0.8371 - val_loss: 0.1063 - val_accuracy: 0.8492
Epoch 62/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1109 - accuracy: 0.8363 - val_loss: 0.1021 - val_accuracy: 0.8504
Epoch 63/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1105 - accuracy: 0.8399 - val_loss: 0.1027 - val_accuracy: 0.8508
Epoch 64/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1110 - accuracy: 0.8419 - val_loss: 0.1047 - val_accuracy: 0.8476
Epoch 65/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1120 - accuracy: 0.8355 - val_loss: 0.1024 - val_accuracy: 0.8516
Epoch 

79/79 [==============================] - 0s 3ms/step - loss: 0.1067 - accuracy: 0.8511 - val_loss: 0.1062 - val_accuracy: 0.8420
Epoch 117/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1075 - accuracy: 0.8511 - val_loss: 0.1060 - val_accuracy: 0.8408
Epoch 118/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1069 - accuracy: 0.8435 - val_loss: 0.1032 - val_accuracy: 0.8472
Epoch 119/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1074 - accuracy: 0.8391 - val_loss: 0.1026 - val_accuracy: 0.8448
Epoch 120/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1075 - accuracy: 0.8423 - val_loss: 0.1022 - val_accuracy: 0.8472
Epoch 121/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1067 - accuracy: 0.8447 - val_loss: 0.1037 - val_accuracy: 0.8436
Epoch 122/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1068 - accuracy: 0.8483 - val_loss: 0.1030 - val_accuracy: 0.8436
Epoch 123/200

79/79 [==============================] - 0s 3ms/step - loss: 0.1034 - accuracy: 0.8523 - val_loss: 0.1034 - val_accuracy: 0.8428
Epoch 174/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1016 - accuracy: 0.8531 - val_loss: 0.1042 - val_accuracy: 0.8424
Epoch 175/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1027 - accuracy: 0.8507 - val_loss: 0.1046 - val_accuracy: 0.8412
Epoch 176/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1012 - accuracy: 0.8511 - val_loss: 0.1054 - val_accuracy: 0.8396
Epoch 177/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1030 - accuracy: 0.8443 - val_loss: 0.1041 - val_accuracy: 0.8428
Epoch 178/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1030 - accuracy: 0.8503 - val_loss: 0.1035 - val_accuracy: 0.8448
Epoch 179/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1035 - accuracy: 0.8431 - val_loss: 0.1066 - val_accuracy: 0.8424
Epoch 180/200

[0.10256778448820114, 0.8479999899864197]
Epoch 1/200
79/79 [==============================] - 1s 5ms/step - loss: 0.1410 - accuracy: 0.7979 - val_loss: 0.1125 - val_accuracy: 0.8360
Epoch 2/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1217 - accuracy: 0.8167 - val_loss: 0.1092 - val_accuracy: 0.8416
Epoch 3/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1241 - accuracy: 0.8135 - val_loss: 0.1070 - val_accuracy: 0.8444
Epoch 4/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1175 - accuracy: 0.8319 - val_loss: 0.1065 - val_accuracy: 0.8500
Epoch 5/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1196 - accuracy: 0.8279 - val_loss: 0.1049 - val_accuracy: 0.8512
Epoch 6/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1164 - accuracy: 0.8303 - val_loss: 0.1052 - val_accuracy: 0.8484
Epoch 7/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1134 - accuracy: 0.8395 - val_loss: 0

79/79 [==============================] - 0s 3ms/step - loss: 0.1133 - accuracy: 0.8347 - val_loss: 0.1027 - val_accuracy: 0.8492
Epoch 59/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1128 - accuracy: 0.8415 - val_loss: 0.1027 - val_accuracy: 0.8468
Epoch 60/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1129 - accuracy: 0.8391 - val_loss: 0.1025 - val_accuracy: 0.8496
Epoch 61/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1129 - accuracy: 0.8379 - val_loss: 0.1023 - val_accuracy: 0.8496
Epoch 62/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1126 - accuracy: 0.8427 - val_loss: 0.1029 - val_accuracy: 0.8484
Epoch 63/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1128 - accuracy: 0.8387 - val_loss: 0.1028 - val_accuracy: 0.8508
Epoch 64/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1132 - accuracy: 0.8355 - val_loss: 0.1033 - val_accuracy: 0.8476
Epoch 65/200
79/79 

Epoch 116/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1065 - accuracy: 0.8415 - val_loss: 0.1047 - val_accuracy: 0.8432
Epoch 117/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1082 - accuracy: 0.8443 - val_loss: 0.1042 - val_accuracy: 0.8432
Epoch 118/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1070 - accuracy: 0.8475 - val_loss: 0.1026 - val_accuracy: 0.8484
Epoch 119/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1059 - accuracy: 0.8463 - val_loss: 0.1047 - val_accuracy: 0.8440
Epoch 120/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1075 - accuracy: 0.8431 - val_loss: 0.1035 - val_accuracy: 0.8420
Epoch 121/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1074 - accuracy: 0.8463 - val_loss: 0.1027 - val_accuracy: 0.8480
Epoch 122/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1070 - accuracy: 0.8455 - val_loss: 0.1023 - val_accuracy: 0.8476

79/79 [==============================] - 0s 3ms/step - loss: 0.1054 - accuracy: 0.8435 - val_loss: 0.1019 - val_accuracy: 0.8480
Epoch 174/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1033 - accuracy: 0.8515 - val_loss: 0.1026 - val_accuracy: 0.8436
Epoch 175/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1044 - accuracy: 0.8443 - val_loss: 0.1027 - val_accuracy: 0.8468
Epoch 176/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1042 - accuracy: 0.8499 - val_loss: 0.1017 - val_accuracy: 0.8460
Epoch 177/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1027 - accuracy: 0.8511 - val_loss: 0.1026 - val_accuracy: 0.8444
Epoch 178/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1043 - accuracy: 0.8443 - val_loss: 0.1019 - val_accuracy: 0.8448
Epoch 179/200
79/79 [==============================] - 0s 3ms/step - loss: 0.1044 - accuracy: 0.8431 - val_loss: 0.1042 - val_accuracy: 0.8476
Epoch 180/200

[0.10553993284702301, 0.8435999751091003]
Epoch 1/200
79/79 [==============================] - 1s 8ms/step - loss: 0.1428 - accuracy: 0.8015 - val_loss: 0.1332 - val_accuracy: 0.8156
Epoch 2/200
79/79 [==============================] - 1s 6ms/step - loss: 0.1240 - accuracy: 0.8171 - val_loss: 0.1190 - val_accuracy: 0.8188
Epoch 3/200
79/79 [==============================] - 0s 6ms/step - loss: 0.1210 - accuracy: 0.8247 - val_loss: 0.1055 - val_accuracy: 0.8508
Epoch 4/200
79/79 [==============================] - 1s 6ms/step - loss: 0.1173 - accuracy: 0.8335 - val_loss: 0.1045 - val_accuracy: 0.8520
Epoch 5/200
79/79 [==============================] - 0s 6ms/step - loss: 0.1225 - accuracy: 0.8187 - val_loss: 0.1072 - val_accuracy: 0.8460
Epoch 6/200
79/79 [==============================] - 0s 6ms/step - loss: 0.1162 - accuracy: 0.8299 - val_loss: 0.1128 - val_accuracy: 0.8232
Epoch 7/200
79/79 [==============================] - 0s 6ms/step - loss: 0.1151 - accuracy: 0.8331 - val_loss: 0

79/79 [==============================] - 1s 7ms/step - loss: 0.1135 - accuracy: 0.8375 - val_loss: 0.1027 - val_accuracy: 0.8496
Epoch 59/200
79/79 [==============================] - 1s 7ms/step - loss: 0.1123 - accuracy: 0.8391 - val_loss: 0.1036 - val_accuracy: 0.8476
Epoch 60/200
79/79 [==============================] - 1s 7ms/step - loss: 0.1128 - accuracy: 0.8387 - val_loss: 0.1026 - val_accuracy: 0.8488
Epoch 61/200
79/79 [==============================] - 1s 6ms/step - loss: 0.1122 - accuracy: 0.8383 - val_loss: 0.1023 - val_accuracy: 0.8488
Epoch 62/200
79/79 [==============================] - 1s 6ms/step - loss: 0.1114 - accuracy: 0.8375 - val_loss: 0.1035 - val_accuracy: 0.8496
Epoch 63/200
79/79 [==============================] - 1s 7ms/step - loss: 0.1120 - accuracy: 0.8439 - val_loss: 0.1028 - val_accuracy: 0.8480
Epoch 64/200
79/79 [==============================] - 1s 6ms/step - loss: 0.1123 - accuracy: 0.8363 - val_loss: 0.1046 - val_accuracy: 0.8468
Epoch 65/200
79/79 

#### Impact of Hyperparameters on Accuracy

In [ ]:
#A very slow learning rate coupled with with a high number of epochs allowed us to achieve very high accuracy


n_layers = 2
n_nodes = 256 #, or 512
learning_rates = 0.000001 #,0.0001
epochs = 200

#### Confusion Matrix

- Precision
- Recall
- F1 score
- Accuracy

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

predictions = (model.predict(X_val) > 0.5).astype("int32")

# Compute confusion matrix
cm = confusion_matrix(y_val, predictions)

# Visualizing the confusion matrix
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()

# Bias Evaluation

https://fairlearn.org/main/user_guide/assessment/common_fairness_metrics.html

#### Demographic parity

"Demographic parity is achieved when the probability of a certain prediction is not dependent on sensitive group membership".

Our data has "sensitive" features; Notably gender and race.